# CNN 모델 구성

## 시뮬레이션 데이터에 대한 CNN 모델 구성

시뮬레이션 데이터 불러오기

In [ ]:
import os
import cv2
X = []
y = []
for item in os.listdir():
    if 'unmasked' not in item: continue
    fig = cv2.imread(item)
    fig = fig[174:174+250, 174:174+250]
    X.append(fig/255)
    y.append(item.split('.')[0].split('_')[0])

In [ ]:
import numpy as np

X = np.array(X)
y = np.array(y, dtype='uint8')
print(X.shape, y.shape)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=X.shape[1:]),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5), loss='mse', metrics=['mae', 'mse'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
history = model.fit(X_train, y_train, epochs=128, batch_size=20, validation_data=[X_test, y_test], callbacks=[early_stopping, reduce_lr])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.ylim([0, 50])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()